# Pittsburgh Crime Dataset Exploration

## Import Libraries

In [1]:
import re
import unicodedata
import pandas as pd
import folium

## Load and Inspect Data

In [2]:
df = pd.read_excel("../data/raw/crime_jan1_oct31_2025.xlsx", engine="openpyxl")
df.head()

,Report_Number,ReportedDate,ReportedTime,Hour,DayofWeek,ReportedMonth,NIBRS_Coded_Offense,NIBRS_Offense_Code,NIBRS_Offense_Category,NIBRS_Offense_Type,NIBRS_Crime_Against,NIBRS_Offense_Grouping,Violation,XCOORD,YCOORD,Zone,Tract,Neighborhood,Block_Address
0,PGHP24000024,2024-01-01,00:31,0,Monday,Jan,13A AGGRAVATED ASSAULT,13A,Assault Offenses,Aggravated Assault,Person,A,18 2718 A1 Strangulation Basic - Applying,-80.0268,40.3964,Zone 6,1919,Brookline,"2800 Block of FITZHUGH WAY Pittsburgh, PA"
1,PGHP24000024,2024-01-01,00:31,0,Monday,Jan,13C INTIMIDATION,13C,Assault Offenses,Intimidation,Person,A,18 2706 A1 Terroristic Threats-General,-80.0268,40.3964,Zone 6,1919,Brookline,"2800 Block of FITZHUGH WAY Pittsburgh, PA"
2,PGHP24000024,2024-01-01,00:31,0,Monday,Jan,90Z ALL OTHER OFFENSES,90Z,All other Offenses,All other Offenses,Group B,B,75 3733 A Fleeing or Attempting To Elude Polic...,-80.0268,40.3964,Zone 6,1919,Brookline,"2800 Block of FITZHUGH WAY Pittsburgh, PA"
3,PGHP24000024,2024-01-01,00:31,0,Monday,Jan,23H ALL OTHER LARCENY,23H,Larceny/Theft Offenses,All Other Larceny,Property,A,18 3921 A Theft by Unlawful Taking-Movable – L...,-80.0268,40.3964,Zone 6,1919,Brookline,"2800 Block of FITZHUGH WAY Pittsburgh, PA"
4,PGHP24000017,2024-01-01,00:21,0,Monday,Jan,9999 Vehicle Offense (Not NIBRS Reportable),999,Not NIBRS Reportable,Not NIBRS Reportable,Group B,B,LO 6 101 Discharge of Firearms Prohibited,-80.0243,40.4582,Zone 1,2107,Manchester,"1200 Block of COLUMBUS AVE Pittsburgh, PA"


## Data Cleaning

### Convert ReportedDate to datetime

In [3]:
df['ReportedDate'] = pd.to_datetime(df['ReportedDate'], errors='coerce')
df = df[df["ReportedDate"].notna()]

### Extract Year, Month

In [4]:
df['Year'] = df['ReportedDate'].dt.year
df['Month'] = df['ReportedDate'].dt.month
df['MonthName'] = df['ReportedDate'].dt.strftime("%b")


### Clean neighborhood

In [5]:
NEIGHBORHOOD_NORMALIZATION_MAP = {
    "spring hill city view": "Spring Hill-City View",
    "spring hill-city view": "Spring Hill-City View",
    "spring hill–city view": "Spring Hill-City View",
    "spring hill—city view": "Spring Hill-City View",

    "lincoln lemington belmar": "Lincoln-Lemington-Belmar",
    "lincoln-lemington-belmar": "Lincoln-Lemington-Belmar",
    "lincoln–lemington–belmar": "Lincoln-Lemington-Belmar",
    "lincoln—lemington—belmar": "Lincoln-Lemington-Belmar",
}

def clean_neighborhood(name):
    if not isinstance(name, str):
        return name

   
    n = unicodedata.normalize("NFKC", name).strip()

    
    n = re.sub(r"[\u2012\u2013\u2014\u2015]", "-", n)

    
    n = re.sub(r"\s*-\s*", "-", n)

    key = n.lower()

  
    if key in NEIGHBORHOOD_NORMALIZATION_MAP:
        return NEIGHBORHOOD_NORMALIZATION_MAP[key]

   
    return " ".join(word.capitalize() for word in n.split())

In [6]:
df["Neighborhood"] = df["Neighborhood"].fillna("Unknown")
df["Neighborhood"] = df["Neighborhood"].apply(clean_neighborhood)

### X and Y COORD

In [7]:
df["XCOORD"] = pd.to_numeric(df.get("XCOORD"), errors="coerce")
df["YCOORD"] = pd.to_numeric(df.get("YCOORD"), errors="coerce")
df[['YCOORD', 'XCOORD']].isna().sum()

YCOORD    632
XCOORD    645
dtype: int64

In [8]:
bad_rows = df[df['YCOORD'].isna() | df['XCOORD'].isna()]
bad_rows.head()

,Report_Number,ReportedDate,ReportedTime,Hour,DayofWeek,ReportedMonth,NIBRS_Coded_Offense,NIBRS_Offense_Code,NIBRS_Offense_Category,NIBRS_Offense_Type,...,Violation,XCOORD,YCOORD,Zone,Tract,Neighborhood,Block_Address,Year,Month,MonthName
88,PGHP24000485,2024-01-01,21:48,21,Monday,Jan,11B FORCIBLE SODOMY,11B,Sex Offenses,Forcible Sodomy,...,18 3121 A1 Rape-Forcible Compulsion - Forcible...,NaN,NaN,Zone 6,NaN,Unknown,NaN,2024,1,Jan
105,PGHP24000489,2024-01-01,22:14,22,Monday,Jan,11C SEXUAL ASSAULT WITH AN OBJECT,11C,Sex Offenses,Sexual Assault With An Object,...,18 3123 A3 IDSI-Unconscious Victim - Sexual As...,NaN,NaN,Zone 4,NaN,Unknown,NaN,2024,1,Jan
185,PGHP23169569,2024-01-02,21:00,21,Tuesday,Jan,11A FORCIBLE RAPE,11A,Sex Offenses,Forcible Rape,...,"18 3124.1 Sexual Assault-Sex Offenses, Rape",NaN,NaN,Zone 3,NaN,Unknown,NaN,2024,1,Jan
215,PGHP24001063,2024-01-03,08:29,8,Wednesday,Jan,36B STATUTORY RAPE,36B,Sex Offenses,Statutory Rape,...,18 3122.1 Statutory Sexual Assault,NaN,NaN,Zone 5,NaN,Unknown,NaN,2024,1,Jan
304,PGHP24001372,2024-01-03,18:34,18,Wednesday,Jan,36B STATUTORY RAPE,36B,Sex Offenses,Statutory Rape,...,18 3122.1 Statutory Sexual Assault,NaN,NaN,Zone 5,NaN,Unknown,NaN,2024,1,Jan


In [9]:
df = df.dropna(subset=['YCOORD', 'XCOORD'])
df[['YCOORD', 'XCOORD']].isna().sum()

YCOORD    0
XCOORD    0
dtype: int64

### NIBRS_Offense_Category

In [10]:
categories = df['NIBRS_Offense_Category'].dropna().unique()
categories

array(['Assault Offenses', 'All other Offenses', 'Larceny/Theft Offenses',
       'Not NIBRS Reportable', 'Destruction/Damage/Vandalism of Property',
       'Fraud Offenses', 'Weapon Law Violations',
       'Drug/Narcotic Offenses', 'Robbery', 'Motor Vehicle Theft',
       'Burglary/Breaking & Entering', 'Arson', 'Human Trafficking',
       'Kidnapping/Abduction', 'Homicide Offenses'], dtype=object)

### NIBRS_Offense_Type

In [11]:
types = df['NIBRS_Offense_Type'].dropna().unique()
types

array(['Aggravated Assault', 'Intimidation', 'All other Offenses',
       'All Other Larceny', 'Not NIBRS Reportable', 'Theft from Building',
       'Drunkenness', 'Disorderly Conduct', 'Trespass of Real Property',
       'Simple Assault', 'Destruction/Damage/Vandalism of Property',
       'Shoplifting', 'Credit Card/ATM Fraud',
       'Theft of Motor Vehicle Parts or Accessories',
       'False Pretenses/Swindle/Confidence Game', 'Weapon Law Violations',
       'Pocket-Picking', 'Drug Equipment Violations',
       'Stolen Property Offenses', 'Drug/Narcotic Violations', 'Robbery',
       'Motor Vehicle Theft', 'Burglary/Breaking & Entering',
       'Driving Under the Influence', 'Theft from Motor Vehicle',
       'Impersonation', 'Bad Checks', 'Counterfeiting/Forgery',
       'Assisting or Promoting Prostitution', 'Arson',
       'Family Offenses, Nonviolent', 'Pornography/Obscene Material',
       'Curfew/Loiting/Vagrancy Violations', 'Purse-snatching',
       'Involuntary Servitude',

### NIBRS Offense Occurences

In [12]:
df['NIBRS_Offense_Category'].value_counts()
df['NIBRS_Offense_Type'].value_counts()

NIBRS_Offense_Type
Not NIBRS Reportable                           13080
Destruction/Damage/Vandalism of Property        5874
Simple Assault                                  5651
Intimidation                                    4382
Drug/Narcotic Violations                        3898
All other Offenses                              3789
Disorderly Conduct                              3645
All Other Larceny                               3153
Theft from Building                             3103
Shoplifting                                     3102
Theft from Motor Vehicle                        2389
Drug Equipment Violations                       2293
Motor Vehicle Theft                             2182
Aggravated Assault                              1717
Trespass of Real Property                       1672
Burglary/Breaking & Entering                    1620
Credit Card/ATM Fraud                           1536
Weapon Law Violations                           1444
False Pretenses/Swindle/Con

### Crime Counts per category

In [13]:
category_counts = (
    df.groupby('NIBRS_Offense_Category')
      .size()
      .reset_index(name='Count')
      .sort_values('Count', ascending=False)
)
category_counts

,NIBRS_Offense_Category,Count
10,Larceny/Theft Offenses,13421
12,Not NIBRS Reportable,13080
2,Assault Offenses,11750
0,All other Offenses,11286
5,Drug/Narcotic Offenses,6191
4,Destruction/Damage/Vandalism of Property,5874
6,Fraud Offenses,3924
11,Motor Vehicle Theft,2182
3,Burglary/Breaking & Entering,1620
14,Weapon Law Violations,1444


### Crime types within each category

In [14]:
type_counts = (
    df.groupby(['NIBRS_Offense_Category', 'NIBRS_Offense_Type'])
      .size()
      .reset_index(name='Count')
      .sort_values('Count', ascending=False)
)
type_counts.head(10)

,NIBRS_Offense_Category,NIBRS_Offense_Type,Count
45,Not NIBRS Reportable,Not NIBRS Reportable,13080
18,Destruction/Damage/Vandalism of Property,Destruction/Damage/Vandalism of Property,5874
16,Assault Offenses,Simple Assault,5651
15,Assault Offenses,Intimidation,4382
20,Drug/Narcotic Offenses,Drug/Narcotic Violations,3898
0,All other Offenses,All other Offenses,3789
4,All other Offenses,Disorderly Conduct,3645
35,Larceny/Theft Offenses,All Other Larceny,3153
40,Larceny/Theft Offenses,Theft from Building,3103
38,Larceny/Theft Offenses,Shoplifting,3102


In [30]:
m = folium.Map(location=[40.44, -79.99], zoom_start=12)
for _, row in df.sample(100).iterrows():
    folium.CircleMarker(
        location=[row['YCOORD'], row['XCOORD']],
        radius=2,
        fill=True,
        fill_color="red",
        color="red",
        opacity=0.6,
    ).add_to(m)

m
